# References

https://www.kaggle.com/code/mtszkw/technical-indicators-for-trading-stocks
https://www.kaggle.com/code/thebrownviking20/everything-you-can-do-with-a-time-series
https://www.diva-portal.org/smash/get/diva2:1775077/FULLTEXT01.pdf
https://pub.towardsai.net/predicting-stock-prices-using-arima-fourier-transforms-and-technical-indicators-with-deep-43a164859683


# Import dataset and create other datasets

In [ ]:
import os
import pandas as pd

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import numpy as np                                    # For matrices!
import matplotlib.pyplot as plt                       # To visualize
from sklearn.linear_model import LinearRegression     # For the regression itself
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.preprocessing import OneHotEncoder       # To convert discrete strings to vectors!
from sklearn.preprocessing import normalize           # For normalizing
import seaborn as sns                                 # For plots

In [ ]:
! pip install gdown

In [ ]:
# Show charts when running kernel
init_notebook_mode(connected=True)

# Change default background color for all visualizations
layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(250,250,250,0.8)')
fig = go.Figure(layout=layout)
templated_fig = pio.to_templated(fig)
pio.templates['my_template'] = templated_fig.layout.template
pio.templates.default = 'my_template'

In [ ]:
# SnP500
! gdown --id 1KveL-W2L6YxO-1NKn0n6FeGTKLWgvAL4
! gdown --id 1zpiNfA7BXiHjqj1tOpNnC3lRdDfZ8kC3
! gdown --id 1zdvABHAhe3pmWE2p4UWUpKJN-1q6fVG7
! gdown --id 1rtaZZ9w4kZwuXqcU2QDZq4rD6CHm7fMW

# Nasdaq
! gdown --id 1NyHRoU2YiaCKIhy7afUTuAmC0haWK1Ny
! gdown --id 1vBAdm8bRRD6jZvRbbQ3q4t5hs19fadke
! gdown --id 1pX1bRRoQyxNty5CF6f3BDDeBVFRAd2iQ
! gdown --id 1ngj9kV06RK2ztU5trUtLKy_q2U9ZpYvZ

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1KveL-W2L6YxO-1NKn0n6FeGTKLWgvAL4
To: /content/ES_continuous_adjusted_1min_2011_2018.txt
100% 158M/158M [00:01<00:00, 89.7MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1zpiNfA7BXiHjqj1tOpNnC3lRdDfZ8kC3
To: /content/ES_continuous_adjusted_5min_2011_2018.txt
100% 31.3M/31.3M [00:00<00:00, 42.8MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warni

In [ ]:
pd.read_csv('ES_continuous_adjusted_1min_2011_2018.txt', encoding='latin-1')
pd.read_csv('ES_continuous_adjusted_5min_2011_2018.txt', encoding='latin-1')
pd.read_csv('ES_continuous_adjusted_30min_2011_2018.txt', encoding='latin-1')
pd.read_csv('ES_continuous_adjusted_1hour_2011_2018.txt', encoding='latin-1')

pd.read_csv('NQ_continuous_adjusted_1min_2011_2018.txt', encoding='latin-1')
pd.read_csv('NQ_continuous_adjusted_5min_2011_2018.txt', encoding='latin-1')
pd.read_csv('NQ_continuous_adjusted_30min_2011_2018.txt', encoding='latin-1')
pd.read_csv('NQ_continuous_adjusted_1_hour2011_2018.txt', encoding='latin-1')

FileNotFoundError: ignored

In [ ]:
snp_raw_data

,DateTime,Open,High,Low,Close,Volume
0,2011-01-02 18:00:00,1056.75,1057.75,1056.00,1057.50,1766
1,2011-01-02 18:01:00,1057.50,1057.75,1057.25,1057.50,259
2,2011-01-02 18:02:00,1057.25,1058.00,1057.25,1057.75,513
3,2011-01-02 18:03:00,1058.00,1058.25,1057.75,1058.00,495
4,2011-01-02 18:04:00,1058.00,1058.50,1058.00,1058.00,489
...,...,...,...,...,...,...
2781098,2018-12-31 16:55:00,2457.75,2458.25,2457.25,2458.25,227
2781099,2018-12-31 16:56:00,2458.00,2458.50,2457.75,2458.25,605
2781100,2018-12-31 16:57:00,2458.00,2458.25,2457.75,2458.25,287
2781101,2018-12-31 16:58:00,2458.00,2458.50,2458.00,2458.25,620


In [ ]:
ndq_raw_data.head()

,DateTime,Open,High,Low,Close,Volume
0,2011-01-02 18:00:00,2361.25,2364.25,2361.25,2364.00,169
1,2011-01-02 18:01:00,2363.25,2364.00,2363.25,2363.50,10
2,2011-01-02 18:02:00,2363.50,2364.50,2363.25,2364.50,71
3,2011-01-02 18:03:00,2364.75,2364.75,2364.00,2364.25,70
4,2011-01-02 18:04:00,2364.50,2365.00,2364.25,2364.25,30
